In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
%matplotlib inline

In [2]:
fp = '../data/full_data.csv'

In [3]:
musics = pd.read_csv(fp).iloc[:, 1:].sort_values(by = 'year')

In [4]:
musics.head()

,album,artist,id,title,year,file,mfcc_0,mfcc_1,mfcc_2,mfcc_3,...,bandwidth,centroid,rolloff,acousticness,danceability,energy,loudness,speechiness,tempo,valence
0,The Sugarhill Gang,The Sugarhill Gang,1FnxNtKli7rcB9IU4J8MEg,Rapper's Delight,1980,./data/Rapper's Delight.mp3,1.890811e-08,-3.804202e-07,-2.639254e-07,-6.431760e-08,...,2932.269422,3271.149826,6835.888420,0.2200,0.905,0.692,-14.143,0.199,111.292,0.943
1,Kurtis Blow,Kurtis Blow,5TvcggVu3s2P1fMAS8BupX,The Breaks,1980,./data/The Breaks.mp3,1.057499e-07,6.933380e-08,-1.520881e-07,-3.618482e-07,...,2574.581462,2369.193568,5234.249962,0.0987,0.910,0.576,-13.256,0.109,112.877,0.907
2,8th Wonder,The Sugarhill Gang,4UCkX8nrBlpxjrrEqtb46a,Apache,1981,./data/Apache.mp3,-1.414954e-07,1.606020e-08,-4.006917e-08,-1.196751e-08,...,2535.741911,2907.329677,5791.839284,0.3320,0.827,0.928,-7.266,0.133,115.023,0.261
3,8th Wonder,The Sugarhill Gang,6m0wOLtrzaNwMofVVJTJqs,8th Wonder,1981,./data/8th Wonder.mp3,3.262833e-07,-1.683564e-08,-7.625627e-08,-1.203945e-07,...,2537.327894,3135.834750,5981.420399,0.2980,0.871,0.937,-8.536,0.195,106.494,0.675
4,Planet Rock,Afrika Bambaataa,3uy0jtkM8QYVTsBazkli1x,Planet Rock,1982,./data/Planet Rock.mp3,-8.671024e-08,7.907207e-08,-5.019072e-08,-2.139210e-08,...,2803.020324,3522.282559,6855.145450,0.0110,0.915,0.807,-9.636,0.130,127.350,0.708


In [47]:
# Looking to see if tempo has anything to do with years (i.e. are faster or slower tempos more prevalent in different
# time periods)?
subsets = musics[['mfcc_0','mfcc_1','mfcc_2','mfcc_3','mfcc_4','mfcc_5','mfcc_6','mfcc_7','mfcc_8','mfcc_9','mfcc_10','mfcc_11']]
mfccs = subsets.to_numpy().tolist()
# Transpose
#mfccs = list(map(list, zip(*mfccs)))
len(mfccs)

96

In [48]:
#Scale them
import sklearn.preprocessing 

scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(mfccs)
mfccs_scaled = scaler.transform(mfccs)

mfccs_scaled

array([[ 0.33918171, -3.05832501, -1.49489614, ...,  0.28666662,
        -0.48497296, -0.35877237],
       [ 0.75924312,  0.73731604, -0.80202625, ...,  0.37689845,
        -0.96328863,  1.06748014],
       [-0.43670435,  0.28772033, -0.10803129, ...,  0.54414513,
         0.18456296,  0.05484957],
       ...,
       [-1.19725127, -0.06603335, -1.0707499 , ..., -0.53467741,
         0.24963307, -1.3221795 ],
       [ 0.7598948 ,  0.0913614 ,  1.12827533, ...,  0.05022379,
         0.17224166,  0.16088721],
       [ 0.62444393,  0.3590464 ,  0.56281946, ...,  0.02416563,
         0.24890875,  0.25084443]])

In [49]:
#Check to make sure they scaled right
mfccs_scaled.mean(axis=0)

array([-8.09537622e-18, -3.23815049e-17, -2.08166817e-17, -5.78241159e-18,
       -1.72026745e-17,  8.67361738e-18,  3.90312782e-18, -6.93889390e-18,
        1.35886672e-17,  1.35163871e-17,  1.09865820e-17,  7.51713506e-18])

In [50]:
mfccs_scaled.std(axis=0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [52]:
features = mfccs_scaled
labels = musics['year'].tolis
labels

0     1980
1     1980
2     1981
3     1981
4     1982
      ... 
79    2019
78    2019
93    2019
81    2020
80    2020
Name: year, Length: 96, dtype: int64

ValueError: Found input variables with inconsistent numbers of samples: [12, 96]